In [1]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

ds = load_dataset('billingsmoore/temp', split='train')

references = ds['en']
small_preds = ds['small_predictions']
base_preds = ds['base_predictions']

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')

/home/j/Documents/MLotsawa/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-29 13:04:39.980437: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-29 13:04:40.087942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748538280.132709    5129 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748538280.145578    5129 cuda_blas.cc:140

In [2]:
ref_code = model.encode(references)

In [3]:
small_pred_code = model.encode(small_preds)

In [4]:
base_preds_code = model.encode(base_preds)

In [13]:
from datasets import Dataset
from sklearn.metrics.pairwise import cosine_similarity

code_ds = {}

code_ds['ref'] = ref_code
code_ds['small'] = small_pred_code
code_ds['base'] = base_preds_code

code_ds = Dataset.from_dict(code_ds)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_sim(example):
    # Ensure vectors are 2D for sklearn's cosine_similarity
    ref = np.array(example['ref']).reshape(1, -1)
    small = np.array(example['small']).reshape(1, -1)
    base = np.array(example['base']).reshape(1, -1)

    small_sim = cosine_similarity(ref, small)[0][0]
    base_sim = cosine_similarity(ref, base)[0][0]

    return {
        'small_sim': small_sim,
        'base_sim': base_sim
    }


In [15]:
code_ds = code_ds.map(get_sim)

Map: 100%|██████████| 100000/100000 [01:10<00:00, 1425.08 examples/s]


In [16]:
import numpy as np

small_mean = np.mean(code_ds['small_sim'])
base_mean = np.mean(code_ds['base_sim'])

print(f'Small: {small_mean}')
print(f'Base: {base_mean}')

Small: 0.6607890299283833
Base: 0.6788237633276554
